In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import warnings

from functools import partial
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, iqr
from tqdm import tqdm_notebook as tqdm
import joblib
%matplotlib inline
import seaborn as sns

sys.path.append('../')
# from src.utils import parallel_apply
# from src.feature_extraction import add_features_in_group

warnings.filterwarnings('ignore')

DIR = '/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/'

In [2]:
#description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'data/application_test.csv'))
installments = pd.read_csv(os.path.join(DIR, 'data/installments_payments.csv'))

In [4]:
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in ['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER',
                   'NUM_INSTALMENT_VERSION',
                   'instalment_paid_late_in_days',
                   'instalment_paid_late',
                     'instalment_paid_over_amount',
                        'instalment_paid_over'
                   ]:
        INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES.append((select, agg))
INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES)]

In [6]:
# installments_ = installments[installments['SK_ID_CURR'].isin(positive_ID[:100])]
installments['instalment_paid_late_in_days'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT'] 
installments['instalment_paid_late'] = (installments['instalment_paid_late_in_days'] > 0).astype(int)
installments['instalment_paid_over_amount'] = installments['AMT_PAYMENT'] - installments['AMT_INSTALMENT']
installments['instalment_paid_over'] = (installments['instalment_paid_over_amount'] > 0).astype(int)  # >=0
installments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,instalment_paid_late_in_days,instalment_paid_late,instalment_paid_over_amount,instalment_paid_over
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,-7.0,0,0.000,0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.0,0,0.000,0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.0,0,0.000,0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,-8.0,0,0.000,0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,17.0,1,-4.455,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN,NaN,0,NaN,0
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN,NaN,0,NaN,0
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN,NaN,0,NaN,0
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN,NaN,0,NaN,0


In [7]:
#features = pd.DataFrame({'SK_ID_CURR':installments['SK_ID_CURR'].unique()})


In [8]:
prev_applications = installments.sort_values(['SK_ID_CURR', 'DAYS_INSTALMENT'])
features_all = pd.DataFrame({'SK_ID_CURR': installments['SK_ID_CURR'].unique()})
groupby_aggregate_names = []
number_of_applications = [1, 3, 5]
#number = 1
for number in number_of_applications:
    for groupby_cols, specs in tqdm(INSTALLMENTS_PAYMENTS_AGGREGATION_RECIPIES):
        group_object = prev_applications.groupby(groupby_cols).tail(number)
        group_object = group_object.groupby(groupby_cols)
        # prev_applications_tail = prev_app_sorted.groupby('SK_ID_CURR').tail(number)
        # tail_groupby = prev_applications_tail.groupby('SK_ID_CURR')
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}_{}_{}'.format('_'.join(groupby_cols), agg, select,'last', number)
            features_all = features_all.merge(group_object[select]
                                .agg(agg)
                                .reset_index()
                                .rename(index=str,
                                        columns={select: groupby_aggregate_name})
                                [groupby_cols + [groupby_aggregate_name]],
                                on=groupby_cols,
                                how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
features_all

,SK_ID_CURR,SK_ID_CURR_mean_AMT_INSTALMENT_last_1,SK_ID_CURR_mean_AMT_PAYMENT_last_1,SK_ID_CURR_mean_DAYS_ENTRY_PAYMENT_last_1,SK_ID_CURR_mean_DAYS_INSTALMENT_last_1,SK_ID_CURR_mean_NUM_INSTALMENT_NUMBER_last_1,SK_ID_CURR_mean_NUM_INSTALMENT_VERSION_last_1,SK_ID_CURR_mean_instalment_paid_late_in_days_last_1,SK_ID_CURR_mean_instalment_paid_late_last_1,SK_ID_CURR_mean_instalment_paid_over_amount_last_1,...,SK_ID_CURR_var_AMT_INSTALMENT_last_5,SK_ID_CURR_var_AMT_PAYMENT_last_5,SK_ID_CURR_var_DAYS_ENTRY_PAYMENT_last_5,SK_ID_CURR_var_DAYS_INSTALMENT_last_5,SK_ID_CURR_var_NUM_INSTALMENT_NUMBER_last_5,SK_ID_CURR_var_NUM_INSTALMENT_VERSION_last_5,SK_ID_CURR_var_instalment_paid_late_in_days_last_5,SK_ID_CURR_var_instalment_paid_late_last_5,SK_ID_CURR_var_instalment_paid_over_amount_last_5,SK_ID_CURR_var_instalment_paid_over_last_5
0,161674,21070.620,21070.620,-62.0,-60.0,8.0,2.0,-2.0,0.0,0.000,...,5.296565e+07,5.296565e+07,954.3,525.200000,159.200000,0.200000,111.700000,0.000000,0.000000,0.0
1,151639,26197.470,26177.895,-13.0,-13.0,26.0,1.0,0.0,0.0,-19.575,...,1.797950e+08,1.796992e+08,559.3,573.800000,1849.300000,0.300000,0.500000,0.200000,76.636125,0.0
2,193053,8393.175,8393.175,-21.0,-14.0,2.0,3.0,-7.0,0.0,0.000,...,1.608458e+08,1.608458e+08,588.0,610.333333,0.333333,0.333333,246.333333,0.333333,0.000000,0.0
3,199697,20899.665,20899.665,-581.0,-574.0,12.0,1.0,-7.0,0.0,0.000,...,7.810378e+07,7.810378e+07,927.7,1821.200000,1675.700000,0.800000,467.500000,0.300000,0.000000,0.0
4,167756,4481.100,4481.100,-771.0,-747.0,11.0,2.0,-24.0,0.0,0.000,...,8.743182e+05,2.386272e+06,694.0,630.000000,0.700000,0.200000,274.000000,0.300000,512423.833567,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339582,427056,6977.925,6977.925,-2884.0,-2878.0,6.0,1.0,-6.0,0.0,0.000,...,4.767761e+01,4.767761e+01,392.0,450.000000,0.500000,0.000000,2.000000,0.000000,0.000000,0.0
339583,434108,28975.770,28975.770,-840.0,-823.0,1.0,2.0,-17.0,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339584,413433,656487.945,656487.945,-614.0,-610.0,1.0,2.0,-4.0,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339585,434445,55493.055,55493.055,-326.0,-319.0,1.0,2.0,-7.0,0.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
features_all.to_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/features_test/installments_features_all.parquet')